<h1>Convert LIRICAL's 384 Phenopackets to GA4GH Version 2 Format</h1>
<p><a href="https://pubmed.ncbi.nlm.nih.gov/32755546/" target="__blank">Robinson PN, et al. (2020) Interpretable Clinical Genomics with a Likelihood Ratio Paradigm. Am J Hum Genet. 2020 Sep 3;107(3):403-417</a> presented 384 phenopackets formated according to the GA4GH Phenopacket Schema version 1. This notebook converts those phenopackets to version 2 of the Phenopacket Schema.</p>
<p>We use the Python json package to input the V1 phenopackets and apply a series of corrections to ensure the V2 phenopackets are correctly formated.</p>

In [19]:
import phenopackets as PPKt
import json
import os
import re
from google import protobuf
from google.protobuf.json_format import MessageToJson
import time
import hpotk
from pyphetools.validation import *
from pyphetools.visualization import *
from pyphetools.creation import Individual
import pyphetools
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.9.4


<h2>Import HPO object</h2>

In [2]:
hpo = hpotk.load_ontology('http://purl.obolibrary.org/obo/hp.json')

### Get all 384 json files

In [3]:
from os import listdir
from os.path import isfile, join
v1dir = "v1ppkts"
jsonfiles = [f for f in listdir(v1dir) if isfile(join(v1dir, f)) and f.endswith("json")]
print(f"Extracted total of {len(jsonfiles)} V1 Phenopacket V1 files")

Extracted total of 384 V1 Phenopacket V1 files


### Create JSON objects from JSON files

In [4]:
def get_json_object(fname, dirname):
    fpath = join(dirname, fname)
    if not isfile(fpath):
        raise FileNotFoundError(f"Could not find {fpath}")
    with open(fpath, "r") as f:
        json_string = f.read()
        json_dict = json.loads(json_string)
        return json_dict

In [5]:
def get_id(json_dict):
    if 'id' not in json_dict:
        raise ValueError("Could not extract id field")
    else:
        return json_dict.get('id')

In [6]:
def validate_age(ageString):
    """
    return true if we can parse age as an ISO 8601 Period
    """
    if ageString == "N/A":
        return False
    # first check if the string contains digits
    _digits = re.compile('\d')
    if not bool(_digits.search(ageString)):
        print(f"invalid ageString {ageString}")
        return False
    # now check that the string conforms to ISO 8601, e.g. P34Y2M
    match = re.match(
        r'P((?P<years>\d+)Y)?((?P<months>\d+)M)?((?P<weeks>\d+)W)?((?P<days>\d+)D)?(T((?P<hours>\d+)H)?((?P<minutes>\d+)M)?((?P<seconds>\d+)S)?)?',
        ageString
    )
    return match

In [7]:
def get_subject(json_dict):
    if 'subject' not in json_dict:
        raise ValueError("Could not extract id field")
    subject = json_dict.get('subject')
    proband_id = subject.get('id', None)
    ageAtCollection = subject.get('ageAtCollection', None)
    invalid_age_strings = { "N/A", "ADULT", "FETUS" }
    if ageAtCollection is not None:
        iso_age = ageAtCollection.get('age', None)
        iso_age = iso_age.upper()
        if not iso_age in invalid_age_strings:
            if not iso_age.startswith("P"): 
                iso_age = "P" + iso_age
            if not validate_age(iso_age):
                print(f"Could not validate age for {proband_id}: \"{iso_age}\", will omit")
                iso_age = None
    else:
        iso_age = None
        
    sex = subject.get('sex', 'UNKNOWN_SEX')
    return proband_id, iso_age, sex
    

In [8]:
def get_phenotypic_features_list(json_dict):
    if 'phenotypicFeatures' not in json_dict:
        raise ValueError("Could not extract phenotypicFeatures field")
    pfeature_list = []
    for pfeat in json_dict.get('phenotypicFeatures'):
        pfeature = PPKt.PhenotypicFeature()
        ontology_term = pfeat.get('type')
        ontology_id = ontology_term.get('id')
        ontology_label = ontology_term.get('label')
        # The following code uses the HPO to update outdated term IDs and labels
        if ontology_id in hpo:
            hpo_term = hpo.get_term(ontology_id)
            ontology_id = hpo_term.identifier
            ontology_label = hpo_term.name
        hpo_term = PPKt.OntologyClass()
        hpo_term.id = ontology_id.value
        hpo_term.label = ontology_label
        pfeature.type.CopyFrom(hpo_term)
        if 'excluded' in pfeat:
            pfeature.excluded = pfeat.excluded
        evidence_list = pfeat.get('evidence')
        # The 384 v1 phenopackets always have exactly one evidence element
        # do not check here, things will crash if we were wrong
        evidence = evidence_list[0]
        evidence_code = evidence.get('evidenceCode')
        evidence_term = PPKt.OntologyClass()
        evidence_term.id = evidence_code.get('id')
        evidence_term.label = evidence_code.get('label')
        evi = PPKt.Evidence()
        evi.evidence_code.CopyFrom(evidence_term)
        reference_json = evidence.get('reference')
        if reference_json is not None:
            ref = PPKt.ExternalReference()
            ref.id = reference_json.get('id')
            ref.description = reference_json.get('description')
            evi.reference.CopyFrom(ref)
        pfeature.evidence.append(evi)
        pfeature_list.append(pfeature)
    return pfeature_list

<h2>Disease</h2>
<p>This is the format of the disease entries in the V1 schema.</p>
<pre>'diseases': [{'term': {'id': 'OMIM:191900', 'label': 'MUCKLE-WELLS SYNDROME; MWS'}}]</pre>

In [9]:
def get_disease(json_dict):
    if not 'diseases' in json_dict:
        raise ValueError("Could not extract disease field")
    disease_list = json_dict.get('diseases')
    if len(disease_list) != 1:
        raise ValueError(f"Got bad number of diseases, {len(disease_list)}")
    disease_object = disease_list[0]
    return disease_object.get('term')

In [10]:
def get_gene(json_dict):
    """
    We expect one and only one gene here
    """
    if not 'genes' in json_dict:
        raise ValueError("Could not extract genes field")
    gene_list = json_dict.get('genes')
    if len(gene_list) != 1:
        raise ValueError(f"Got bad number of genes, {len(gene_list)}")
    return gene_list[0]

<h2>Variants in v1 phenopacket</h2>
<p>This is the format of the variants entries in the V1 schema.</p>

<pre>
 'variants': [{'vcfAllele': {'genomeAssembly': 'GRCh37', 'chr': '1', 'pos': 247587794, 'ref': 'C', 'alt': 'T'}, 
    'zygosity': {'id': 'GENO:0000135', 'label': 'heterozygous'}}]
 </pre>

In [11]:
def get_one_ga4gh_interpretation(vcf_allele, zygosity, gene, disease, var_id):
    vdescriptor = PPKt.VariationDescriptor()
    vdescriptor.id = var_id
    vdescriptor.gene_context.value_id = gene.get('id')
    vdescriptor.gene_context.symbol = gene.get('symbol')
    vdescriptor.molecule_context =  PPKt.MoleculeContext.genomic
    vdescriptor.allelic_state.id = zygosity.get('id')
    vdescriptor.allelic_state.label = zygosity.get('label')
    vinterpretation = PPKt.VariantInterpretation() 
    vcf_record = PPKt.VcfRecord()
    vcf_record.genome_assembly =  vcf_allele.get('genomeAssembly')
    vcf_record.chrom = vcf_allele.get('chr')
    vcf_record.pos = vcf_allele.get('pos')
    vcf_record.ref = vcf_allele.get('ref')
    vcf_record.alt = vcf_allele.get('alt')
    vdescriptor.vcf_record.CopyFrom(vcf_record)
    vinterpretation.variation_descriptor.CopyFrom(vdescriptor)
    return vinterpretation

In [12]:
def get_interpretation(json_dict, gene, disease, individual_id):
    if not 'variants' in json_dict:
        raise ValueError("Could not find variants")
    ga4gh_var_list = []
    i = 0
    for v in json_dict.get('variants'):
        if 'vcfAllele' not in v:
            raise ValueError("Malformed variant")
        if 'zygosity' not in v:
            raise ValueError("Malformed variant - no zygosity")
        vcf_allele = v.get('vcfAllele')
        zygosity = v.get('zygosity')
        i += 1
        var_id = f"variant-{i}"
        ga4gh_int = get_one_ga4gh_interpretation(vcf_allele, zygosity, gene, disease, var_id)
        ga4gh_var_list.append(ga4gh_int)
    interpretation = PPKt.Interpretation()
    interpretation.id = "interpretation-id"
    interpretation.progress_status = PPKt.Interpretation.ProgressStatus.SOLVED
    interpretation.diagnosis.disease.id = disease.get('id')
    interpretation.diagnosis.disease.label = disease.get('label')
    for var in ga4gh_var_list:
        genomic_interpretation = PPKt.GenomicInterpretation()
        genomic_interpretation.subject_or_biosample_id = individual_id
        # by assumption, variants passed to this package are all causative
        genomic_interpretation.interpretation_status = PPKt.GenomicInterpretation.InterpretationStatus.CAUSATIVE
        genomic_interpretation.variant_interpretation.CopyFrom(var)
        interpretation.diagnosis.genomic_interpretations.append(genomic_interpretation)
    return interpretation  

<h2>Metadata (V1)</h2>

In [13]:
def get_metadata(json_dict):
    if not 'metaData' in json_dict:
        raise ValueError("Could not find metaData")
    metadata = json_dict.get('metaData')
    createdBy = metadata.get('createdBy', 'n/a')
    submittedBy = metadata.get('submittedBy')
    ga4gh_metadata = PPKt.MetaData()
    resources = []
    for res in metadata.get('resources'):
        if res.get('id') == 'ncbitaxon':
            continue # superfluous to add taxon H. sapiens in thius context and we do not have iriPREFIX for this resource
        ga4gh_resource = PPKt.Resource()
        ga4gh_resource.id = res.get('id')
        ga4gh_resource.name = res.get('name')
        ga4gh_resource.url = res.get('url')
        ga4gh_resource.version = res.get('version', 'n/a')
        ga4gh_resource.namespace_prefix = res.get('namespacePrefix')
        ga4gh_resource.iri_prefix = res.get('iriPrefix', 'n/a')
        ga4gh_metadata.resources.append(ga4gh_resource)
    ga4gh_metadata.created_by = createdBy
    if submittedBy is not None:
        ga4gh_metadata.submitted_by = submittedBy
    now = time.time()
    seconds = int(now)
    nanos = int((now - seconds) * 10 ** 9)
    timestamp = protobuf.timestamp_pb2.Timestamp(seconds=seconds, nanos=nanos)
    ga4gh_metadata.created.CopyFrom(timestamp)
    ga4gh_metadata.phenopacket_schema_version = "2.0"
    for res in resources:
        metadata.resources.append(res)
    return ga4gh_metadata

In [14]:
def fix_redundancies(hpo_term_ids):
    """
    removes general ancestors if the theyhave a specific descendent germ in the same phenopacket.
    """
    redundant_term_d = {}
    hpo_term_id_set = set(hpo_term_ids)
    if len(hpo_term_id_set) != len(hpo_term_ids):
        print("found duplicates")
    for term_id1 in hpo_term_id_set:
        for term_id2 in hpo_term_id_set:
            # The ancestor, e.g. Seizure comes first, the other term, e.g. Clonic seizure, second
            # in the following function call
            if hpo.graph.is_ancestor_of(term_id2, term_id1):
                redundant_term_d[term_id2] = term_id1
    # When we get here, we have scanned all terms for redundant ancestors
    non_redundant_terms = set([ tid for tid in hpo_term_ids if tid not in redundant_term_d])
    #if len(redundant_term_d) > 0:
    #    for term, descendant in redundant_term_d.items():
    #        print(f"Removing redundant_term {term} because of descendent {descendant}")
    return non_redundant_terms


def get_clean_terms(pfeatures_list):
    """
    remove terms that are redundant because there is a more specific descendent term
    """
    observed_terms = [t for t in pfeatures_list if not t.excluded]
    excluded_terms = [t for t in pfeatures_list if not t.excluded]
    observed_d = {t.type.id:t for t in observed_terms}
    excluded_d = {t.type.id:t for t in excluded_terms}
    
    observed_ids = fix_redundancies(observed_d.keys())
    excluded_ids = fix_redundancies(excluded_d.keys())
    clean_terms = []
    # this will remove duplicates
    seen_tids = set()
    for tid in observed_ids:
        if tid in seen_tids:
            continue
        else:
            seen_tids.add(tid)
        clean_terms.append(observed_d.get(tid))
    for tid in excluded_ids:
        if tid in seen_tids:
            continue
        else:
            seen_tids.add(tid)
        clean_terms.append(excluded_d.get(tid))    
    return clean_terms
    

In [15]:
def construct_v2_phenopacket(json_dict):
    id = get_id(json_dict)
    proband_id, iso_age, sex= get_subject(json_dict)
    pfeatures_list = get_phenotypic_features_list(json_dict)
    pfeatures_list = get_clean_terms(pfeatures_list)
    gene = get_gene(json_dict)
    disease = get_disease(json_dict)
    interpretation = get_interpretation(json_dict, gene, disease, proband_id)
    # Create phenopacket
    phenopacket = PPKt.Phenopacket()
    phenopacket.id = id
    proband = PPKt.Individual()
    proband.id = proband_id
    if sex == "MALE":
        proband.sex  = PPKt.Sex.MALE
    elif sex == "FEMALE":
        proband.sex = PPKt.Sex.FEMALE
    else:
        proband.sex  = PPKt.Sex.UNKNOWN_SEX
    if iso_age is not None:
        proband.time_at_last_encounter.age.iso8601duration = iso_age
    phenopacket.subject.CopyFrom(proband)
    for pf in pfeatures_list:
        phenopacket.phenotypic_features.append(pf)
    phenopacket.interpretations.append(interpretation)
    metadata = get_metadata(json_dict)
    phenopacket.meta_data.CopyFrom(metadata)
    return phenopacket
    

In [16]:
def output_phenopacket(json_dict, output_dir):
    ppack = construct_v2_phenopacket(json_dict)
    json_string = MessageToJson(ppack)
    pp_id = ppack.id
    fname = pp_id.replace(" ", "").replace(":", "_")
    fname = re.sub('[^A-Za-z0-9_-]', '', fname)  # remove any illegal characters from filename
    fname = fname.replace(" ", "_") + ".json"
    outpth = os.path.join(output_dir, fname)
    with open(outpth, "wt") as fh:
        fh.write(json_string)

<h2>Output</h2>
<p>Here we output the 384 phenopackets in version 2 format. Some of the original files do not have a valid age string, and so they produce a warning during the output. This can be ignored, the corresponding cases our output without an age (which is not required).

In [17]:
out_dir = "v2phenopackets"
v1dir = "v1ppkts"
os.makedirs(out_dir, exist_ok=True)
n = 0
for json_file in jsonfiles:
    json_dict = get_json_object(json_file, v1dir)
    output_phenopacket(json_dict, out_dir)
    print(".", end="")
    n += 1
print(f"\nOutput {n} phenopackets")

................................................................................................................................................................................................................................................................................................................................................................................................
Output 384 phenopackets


<h2>Additional validation</h2>
<p>In the original phenopackets used for the LIRICAL publication, the original cases reports conmtained some redundant HPO terms.
<p>This can be detected using <a href="http://phenopackets.org/phenopacket-tools/stable/cli.html">phenopacket tools</a>.
<p>We have removed duplicated and redundant terms using the above functions and the following call showed no errors</p>
<pre>pxf validate --hpo hp.json *.json</pre>

In [21]:
ingestor = PhenopacketIngestor(indir="v2phenopackets")
ppkt_d = ingestor.get_phenopacket_dictionary()
individuals = [Individual.from_ga4gh_phenopacket(ppkt) for ppkt in ppkt_d.values()]

[pyphetools] Ingested 384 GA4GH phenopackets.


In [22]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo, min_hpo=1)

In [24]:
from IPython.display import HTML, display
qc = QcVisualizer(cvalidator)
display(HTML(qc.to_html()))